# Deploying your trained model

Once you've managed to run your full ML pipeline, you should end up with a registered versions of your model. You can view the different versions of the model in the Azure ML interface in the *Models* section. Of course, now we have a trained model, we also want to actually deploy our model. 

There are different approaches for deploying models, including:

* Using the GUI (selecting a model > clicking on deploy)
* Using the Python SDK

In this notebook, we aim to deploy the model using the SDK. (This will also allow us to automate the deployment as part of our CI/CD pipeline later.)

## Fetching the model

Let's start by fetching the model from our workspace. First, we need to get our workspace again:

In [ ]:
from azureml.core import Workspace

workspace = Workspace.from_config()

After which, we can fetch our model using the *Model* class:

In [ ]:
from azureml.core.model import Model

model_name = "MY_MODEL_NAME"

# Note: Include version parameter to fetch a specific version. 
model = Model(workspace=workspace, name=model_name)

## Deploying the webservice

Now we have our model, we can start deploying it as a webservice. 

To do so, we first need to define the environment in which to run the model using an instance of the [InferenceConfig](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py) class:

In [ ]:
from pathlib import Path

from azureml.core import Environment
from azureml.core.model import InferenceConfig

# Define model environment.
model_dir = Path("../model")
model_env = Environment.from_conda_specification(f"{model_name}-env", model_dir / "environment.yml")
 
# Define inference config (= how to serve model).
inference_config = InferenceConfig(
    entry_script="scripts/serve.py", 
    source_directory=model_dir, 
    environment=env
)

This *InferenceConfig* is similar to the *RunConfig* used for our pipeline steps, in the sense that this config defines the environment to run in, together with the entrypoint to use for serving the model. Note that the serving entrypoint (*serve.py*) defines how your model responds to requests. This file needs to have a specific structure, on which you can find more details [here](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py).

To actually deploy the webservice, you can use the *Model.deploy* method:

In [ ]:
service_name = "MY_WEBSERVICE_NAME"

# Remove any existing service under the same name.
try:
    existing_service = Webservice(workspace, name=service_name)
    print("WARNING: Deleting existing service '%s'", service_name)
    existing_service.delete()
except WebserviceException:
    pass

# Actually deploy the model.
print("Starting deployment")
service = Model.deploy(workspace, service_name, [model], inference_config)

# Wait for deployment to complete.
service.wait_for_deployment(show_output=True)

After waiting for a moment, this should result in a deployed version of your model with a (publicly) accessible end point>?

### Exercise 

Try to query your webservice using the requests library.

In [ ]:
%load answers/webservice.py